# Proving the Playpen Agent/CodePlan for Compiler Errors

We need prove that the playpen code plan agents workflow will work as expected. This will be a way to run them and make sure that things work as we expect.

## Setup

In [ ]:
import pprint

pp = pprint.PrettyPrinter(indent=2)

## Create the Test Environment

For creating the test environment we will:

1. move the coolstore to a temp direcotry
2. Update the InventoryEntity to use UUID for the itemID as the type.

In [ ]:
import tempfile
import shutil
import os
import subprocess

#First we need to download coolstore if it does not exist.

if not os.path.exists("./coolstore"):
    subprocess.run("../../example/fetch.sh")

if not os.path.exists("../../kai-analyzer/kai-analyzer"):
    subprocess.run(["go", "build", "-o", "kai-analyzer", "main.go"], cwd="../../kai-analyzer")

temp_dir = tempfile.TemporaryDirectory()
coolstore_path = os.path.join(temp_dir.name, "coolstore")
shutil.copytree("./coolstore", coolstore_path)

## make the necessary change

print(temp_dir)


## Set up the Code Plan types and run code plan.

In [ ]:
from pathlib import Path
from playpen.repo_level_awareness.api import RpcClientConfig
from playpen.repo_level_awareness.codeplan import TaskManager
from playpen.repo_level_awareness.task_runner.analyzer_lsp.validator import AnlayzerLSPStep
from playpen.repo_level_awareness.task_runner.analyzer_lsp.task_runner import AnalyzerTaskRunner
from kai.service.llm_interfacing.model_provider import ModelProvider
from kai.models.kai_config import KaiConfig
from playpen.repo_level_awareness.vfs.git_vfs import RepoContextManager
import logging

logging.basicConfig(level=logging.DEBUG)
logging.getLogger('httpx').setLevel(logging.WARNING)
logging.getLogger('httpcore').setLevel(logging.WARNING)
logging.getLogger('genai.extensions.langchain.chat_llm').setLevel(logging.WARNING)

config = RpcClientConfig(Path(coolstore_path),
                         "../../kai-analyzer/kai-analyzer",
                         "/home/fabian/projects/github.com/konveyor/rulesets/default/generated",
                         Path("/home/fabian/fabian/.vscode/extensions/redhat.java-1.30.0-linux-x64/server/bin/jdtls"),
                         Path("./java-bundle/java-analyzer-bundle.core-1.0.0-SNAPSHOT.jar"),
                         "konveyor.io/target=quarkus",
                         None,
                         None)

kai_config = KaiConfig.model_validate_filepath("../../kai/config.toml")
modelProvider = ModelProvider(kai_config.models)
rcm = RepoContextManager(config.repo_directory, modelProvider.llm)


agent = AnalyzerTaskRunner(modelProvider.llm)

# TODO: Use seed_tasks argument to supply initial task to the task_manager
task_manager = TaskManager(
        config,
        rcm,
        None,
        # TODO: Set up with maven as well?
        validators=[AnlayzerLSPStep(config)],
        agents=[agent],
    )


# TODO: Make this get_next_task(max_priority=0) to only grab seeded tasks
# Can also do: 
#   initial_task = task_manager.get_next_task(max_priority=0)
#   ...
#   task_manager.supply_result(task_manager.execute_task(initial_task)
#   follow_up_task = task_manager.get_next_task(max_priority=0)
#   # do whatever to show what followup is
#   task_manager.supply_result(task_manager.execute_task(follow_up_task)
# etc ...
# can  introspect the stack using task_manager.task_stacks
# priority 0 tasks will be accessible with task_manager.task_stacks.get(0)
# So can see all the new tasks that are spawned and how that stack changes as we progress
# as well as showing all the tasks in the stack if we want to show all the work that's been detected in general
# Can do that with task_manager.task_stacks.values() -> list of lists of tasks associated with each priority level
for task in task_manager.get_next_task():
    task_manager.supply_result(task_manager.execute_task(task))

## Cleanup temp dir

In [ ]:
temp_dir.cleanup()